# 🔄 Inferência Recorrente (Batch Scoring)

## 🎯 Objetivo
Gerar previsões para os próximos dias (Forecast Horizon) usando o modelo produtivo.

## ⚙️ Fluxo
1.  **Carrega Contexto**: Lê os últimos 90 dias.
2.  **Carrega Modelo**: Baixa a versão `Champion`.
3.  **Previsão**: Gera e persiste as previsões.

---

# 🔄 Inferência Recorrente de Vendas - CVC Lojas

## 🎯 Objetivo
Execução periódica (Semanal/Mensal) para gerar novas previsões de vendas.
Este notebook não treina modelos. Ele carrega o modelo produtivo (`All-in-One`) e gera forecast baseando-se no histórico mais recente.

## ⚙️ Fluxo de Execução
1.  **Context Loading:** Carrega os últimos 90 dias de vendas (Janela de Contexto) do Data Lake.
2.  **Model Loading:** Baixa o modelo do Unity Catalog (`Usage: Production`).
3.  **Inference:** O Wrapper `UnifiedForecaster` recebe o contexto, normaliza, prevê e desnormaliza.
4.  **Persistence:** Salva os resultados na tabela `bip_vprevisao_lojas_futuro`.


In [0]:
# Importação de bibliotecas essenciais
# --- SETUP INICIAL ---
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.getcwd())

from src.validation.config import Config
from src.validation.data import DataIngestion
from datetime import timedelta, date
import mlflow
from mlflow.tracking import MlflowClient
import pyspark.sql.functions as F
import pandas as pd
from datetime import datetime
import numpy as np
import pandas as pd
import pyspark.sql.functions as F

# Darts classes para Wrapper funcionar
from darts import TimeSeries

# Configs Spark
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")
client = MlflowClient()

In [0]:
# --- 1. DEFINIÇÃO DA JANELA DE CONTEXTO ---
# Simulando "Hoje" (em prod usar date.today())
today = date.today()
#today = datetime.strptime("2025-02-01", "%Y-%m-%d").date() 

# Janela de Lookback: precisamos de histórico suficiente para os Lags do modelo (ex: 60-90 dias)
context_days = 90 
start_context = today - timedelta(days=context_days)

# Config Dinâmica
config = Config(spark)
config.DATA_START = start_context.strftime("%Y-%m-%d")
config.INGESTION_END = today.strftime("%Y-%m-%d")
config.SCHEMA = "cvc_pred"

print(f"📅 Data de Referência (Hoje): {today}")
print(f"🔎 Carregando contexto a partir de: {config.DATA_START}")

In [0]:
# --- 2. CARREGAMENTO DO CONTEXTO (SPARK) ---
# Reutilizamos a classe DataIngestion para garantir consistência nas features
ingestion = DataIngestion(spark, config)

print("   ⏳ Lendo dados históricos recentes...")
df_context_spark = ingestion.create_training_set()

# Filtro de Segurança e Seleção de Colunas
df_context_spark = df_context_spark.filter(
    F.col("DATA").between(config.DATA_START, config.INGESTION_END)
)

# Traz para Pandas (Driver) - Volume pequeno pois é só janela recente
df_context_pd = df_context_spark.toPandas()
df_context_pd['data'] = pd.to_datetime(df_context_pd['data'])

# Injeta parametro 'n' para o wrapper saber o horizonte desejado
FORECAST_HORIZON = 35
df_context_pd['n'] = FORECAST_HORIZON

In [0]:
# Carrega o modelo validado do registro
# --- 3. CARREGAMENTO DO MODELO ---
model_name = f"{config.CATALOG}.{config.SCHEMA}.cvc_lojas_forecast_production"
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}@Champion")
print("✅ Modelo Carregado!")
mv = client.get_model_version_by_alias(name=model_name, alias="Champion")

print("Modelo:", model_name)
print("Versão do modelo:", mv.version)
print("Run ID:", mv.run_id)
print("Current stage:", mv.current_stage)
print("Description:", mv.description)

In [0]:
# --- PREPARAÇÃO DE DADOS DE MERCADO ---

# 1. Carrega a tabela de suporte (a mesma usada no treino)
df_market_spark = spark.table(f"{config.CATALOG}.{config.SCHEMA}.historico_suporte_loja")

# 2. Pivota para criar as colunas (IPCA, DOLAR...)
# Importante: O nome das colunas deve bater exatamente com o treino
df_market_wide = (df_market_spark
    .groupBy("data")
    .pivot("metricas")
    .agg(F.sum("valor"))
    .na.fill(0.0))

# 3. Converte para Pandas para fazer o merge local (já que a inferência é pandas)
pdf_market = df_market_wide.toPandas()
pdf_market['data'] = pd.to_datetime(pdf_market['data']).dt.strftime('%Y-%m-%d')

In [0]:
# --- MONTAGEM FINAL DO CONTEXTO (MERCADO + FUTURO) COM AJUSTE DE SCHEMA MLFLOW ---

# ==============================================================================
# 1. PREPARAÇÃO DE DADOS DE MERCADO (Suporte Global)
# ==============================================================================
print("📊 Carregando dados de mercado...")

df_market_spark = spark.table(f"{config.CATALOG}.{config.SCHEMA}.historico_suporte_loja")
df_market_wide = (df_market_spark
    .groupBy("data")
    .pivot("metricas")
    .agg(F.sum("valor"))
    .na.fill(0.0)
)

pdf_market = df_market_wide.toPandas()
pdf_market['data'] = pd.to_datetime(pdf_market['data'])

# ==============================================================================
# 2. CRIAÇÃO DO ESQUELETO EXPANDIDO (±3 meses para satisfazer o modelo)
# ==============================================================================
print("⏳ Montando timeline completa com margem de segurança...")

df_context_pd['data'] = pd.to_datetime(df_context_pd['data'])
last_date_history = df_context_pd['data'].max()
start_date_context = df_context_pd['data'].min()

# Margem de 3 meses para trás conforme definido no treinamento (data.py)
safe_start_date = start_date_context - pd.DateOffset(months=3)
future_dates = pd.date_range(start=last_date_history + pd.Timedelta(days=1), periods=FORECAST_HORIZON + 15, freq='D')

# Range total para evitar erros de dimensão/quarter
inference_range = pd.date_range(start=safe_start_date, end=future_dates.max(), freq='D')

static_cols = ['codigo_loja', 'cluster_loja', 'sigla_uf', 'tipo_loja', 'modelo_loja']
static_cols = [c for c in static_cols if c in df_context_pd.columns]
df_stores_reference = df_context_pd.sort_values('data').groupby('codigo_loja')[static_cols].tail(1)

df_full_timeline = df_stores_reference.assign(key=1).merge(
    pd.DataFrame({'data': inference_range, 'key': 1}), 
    on='key'
).drop('key', axis=1)

# Merge com histórico DE VENDAS (Target) - Feriado pegaremos da tabela cheia
# Primeiro garantimos que os IDs estejam limpos para o merge funcionar
df_full_timeline['codigo_loja'] = df_full_timeline['codigo_loja'].astype(str).str.replace(r'\.0$', '', regex=True)
df_context_pd['codigo_loja'] = df_context_pd['codigo_loja'].astype(str).str.replace(r'\.0$', '', regex=True)

df_full_timeline = pd.merge(
    df_full_timeline, 
    df_context_pd[['data', 'codigo_loja', 'target_vendas']], 
    on=['data', 'codigo_loja'], 
    how='left'
)

# --- CORREÇÃO: Carrega Calendário de Feriados COMPLETO (Passado + Futuro) ---
print("🗓️ Carregando calendário oficial de feriados...")
df_feriados_spark = spark.table(f"{config.CATALOG}.{config.SCHEMA}.historico_feriados_loja")

# Prepara dataframe de feriados
df_feriados_pd = (df_feriados_spark
    .select("codigo_loja", "data", "valor")
    .withColumn("codigo_loja", F.col("codigo_loja").cast("string"))
    .toPandas()
)
df_feriados_pd['data'] = pd.to_datetime(df_feriados_pd['data'])
df_feriados_pd.rename(columns={'valor': 'is_feriado'}, inplace=True)

# Garante limpeza do ID de Feriados
df_feriados_pd['codigo_loja'] = df_feriados_pd['codigo_loja'].astype(str).str.replace(r'\.0$', '', regex=True)

# Merge de Feriados na Timeline Principal
df_full_timeline = pd.merge(
    df_full_timeline,
    df_feriados_pd,
    on=['data', 'codigo_loja'],
    how='left'
)

# Preenchimento de nulos
df_full_timeline['target_vendas'] = df_full_timeline['target_vendas'].fillna(0.0)
df_full_timeline['is_feriado'] = df_full_timeline['is_feriado'].fillna(0.0)
df_full_timeline['n'] = int(FORECAST_HORIZON)

# ==============================================================================
# 3. MERGE FINAL E COMPATIBILIZAÇÃO DE SCHEMA (MLFLOW)
# ==============================================================================

df_inference_final = pd.merge(df_full_timeline, pdf_market, on='data', how='left')

# Preenchimento de indicadores de mercado (ffill/bfill para garantir a margem de 3 meses)
cols_mercado = [c for c in pdf_market.columns if c != 'data']
df_inference_final[cols_mercado] = df_inference_final[cols_mercado].ffill().bfill().fillna(0.0)

# 1. Limpeza de ID da Loja
df_inference_final['codigo_loja'] = (
    df_inference_final['codigo_loja']
    .astype(str)
    .str.replace(r'\.0$', '', regex=True)
)

# 2. Tipos Numéricos (Garante Double/Float conforme exigido pelo MLflow)
for col in cols_mercado + ['target_vendas', 'is_feriado']:
    df_inference_final[col] = df_inference_final[col].astype(float)

# 3. Ajuste de tipos Long/Integer
df_inference_final['n'] = df_inference_final['n'].astype(int)

# 4. CONVERSÃO DE DATA PARA STRING (Exigência do Schema do MLflow: 'data': string)
# Fazemos isso por último para garantir que o Darts receba o formato correto via Wrapper
df_inference_final['data'] = df_inference_final['data'].dt.strftime('%Y-%m-%d')

print(f"✅ Tratamento concluído. Schema compatibilizado com MLflow.")
print(f"   Colunas totais: {len(df_inference_final.columns)} | Linhas: {len(df_inference_final)}")

In [0]:
# Executa a inferência (Forecast)
# --- PREVISÃO COM SANEAMENTO ATÔMICO ---
print("🔮 Saneando dados e gerando previsões...")

# 1. Executa o Saneamento Atômico (Garante colunas 1D puras)
clean_dict = {}
for col in df_inference_final.columns.unique():
    col_name = str(col).strip()
    series_data = df_inference_final[col]
    # Se houver colunas duplicadas vindas do merge, pega a primeira
    if isinstance(series_data, pd.DataFrame):
        series_data = series_data.iloc[:, 0]
    clean_dict[col_name] = series_data.values.flatten()

df_inference_cleaned = pd.DataFrame(clean_dict)

# 2. Chama o modelo usando o DataFrame limpo
forecast_df = loaded_model.predict(df_inference_cleaned)

# 3. Adiciona metadados de rastreabilidade
forecast_df['version_model'] = mv.version
forecast_df['description_model'] = mv.description
forecast_df['model_name'] = model_name
forecast_df['data_reference'] = datetime.now()

print(f"✅ Previsão concluída para {forecast_df['codigo_loja'].nunique()} lojas.")

In [0]:
# --- 5. PERSISTÊNCIA (WRITE BACK) ---
output_table = f"{config.CATALOG}.{config.SCHEMA}.previsao_lojas_futuro"

print(f"💾 Salvando resultados em: {output_table}")
(spark.createDataFrame(forecast_df)
 .write
 .format("delta")
 .mode("append") # Append histórico de previsões
 .option("mergeSchema", "true")
 .saveAsTable(output_table)
)
spark.sql(f"OPTIMIZE {output_table}")
print("✨ Sucesso! Dados salvos e otimizados.")

In [0]:
display(forecast_df)